In [1]:
import numpy as np

def pseudo_mask(length, alpha=1.5, u_range=(0, 1)):
    """
        Generate a pseudo-random mask based on pseudo-random integer sequence
        a_i = ceiling(α(i + u)), α ∈ (1, 2), with some u ∈ (0, 1).
        length: length of the sequence
        alpha: alpha value'
        u_range: range of u
        returns mask of items to keep as 0 or 1 at each index
        from paper: https://arxiv.org/abs/1412.6071
    """
    u = np.random.uniform(u_range[0],u_range[1])  # U value
    sequence = np.zeros(length, dtype=int)
    for i in range(length):
        a = np.ceil(alpha * (i + u)).astype(int)
        if a >= length:
            break
        sequence[a] = 1
    return np.array(sequence)

def perf_conv1d(U, K, prob=1.0, padding=0, stride=1, 
                dilation=1, type='uniform', alpha=1.5, 
                u_range=(0, 1)):
    """
        Convolution with perforation from paper https://arxiv.org/abs/1504.08362
        U: input signal
        K: kernel
        prob: probability of keeping the element in the mask
        padding: padding size
        stride: stride size
        dilation: dilation size
        type: uniform or pseudo
        alpha: alpha value for pseudo mask
        u_range: range of u for pseudo mask
    """
    U = np.array(U) 
    K = np.array(K) 
    if dilation > 1:
        K = np.insert(K[:-1], 
             np.repeat(np.arange(len(K)-1), dilation-1), 0)
    U = np.pad(U, (padding,padding), 'constant')
    d = len(K)
    n = len(U)
    # Mask M
    M = np.array([U[i:i+d] for i in range(0, n-d+1, stride)]) 
    if type=='uniform':
        # Indicies in M to keep
        I = np.random.choice([0, 1], 
            size=M.shape[0], p=[1-prob, prob]) 
    else:
        I = pseudo_mask(M.shape[0], alpha, u_range)
    # perforate M
    M = M*I[:, np.newaxis] 
    # Convolution with perforation
    V_hat = np.array([np.sum(m*K) for m in M]) 
    # replace 0s with nearest non-zero element
    non_zero_i = np.where(V_hat != 0)[0]
    for i in range(len(V_hat)):
        nearest_i = non_zero_i[np.argmin(np.abs(non_zero_i - i))]
        V_hat[i] = V_hat[nearest_i]
    return np.array(V_hat)

def conv1d(U, K, padding=0, stride=1, dilation=1):
    U = np.array(U) 
    K = np.array(K) 
    if dilation > 1:
        K = np.insert(K[:-1], np.repeat(np.arange(len(K)-1), dilation-1), 0)
    U = np.pad(U, (padding,padding), 'constant')
    d = len(K)
    n = len(U)
    M = np.array([U[i:i+d] for i in range(0, n-d+1, stride)]) # Mask M
    return M@K.T

U = np.arange(1,21)
K = np.array([1, 1, 1])
padding = 0
stride = 1
dilation = 1
V = conv1d(U, K, padding=padding, stride=stride, dilation=dilation)
print("input:", U, "with length", len(U))
print("kernel:", K, "with length", len(K))
print('Conv1d output V:', V, "with length", len(V))
V_hat = perf_conv1d(U, K, prob=0.2, padding=padding, stride=stride, dilation=dilation, 
                    type='pseudo', alpha=1.5, u_range=(0, 1))
print('Perforated Conv1d output V_hat:', V_hat, "with length", len(V_hat))
